## Introduction to `igraph`

### Installing `igraph`

The simplest way to install `igraph` is via `pip`, just like any other Python library:

```bash
$ pip install igraph
```

If you use [conda](https://docs.continuum.io/anaconda/), you can install `igraph` from `conda-forge` as follows:

```bash
$ conda install -c conda-forge python-igraph
```

Even though the core of `igraph` is implemented in C and C++, the Python Package Index hosts pre-compiled binaries for most platforms, so the command above should just work on the vast majority of systems. If you are experiencing problems with the installation or you do not want to install `igraph` on your computer yet, you can also try it on [Google Colab](https://colab.research.google.com) by opening a new notebook and typing:

```
!pip install igraph
```

Alternative installation options, including installation into virtual environments (in case you do not have administrator access on your computer or you do not want to install arbitrary packages into the system Python) are listed in the [Installation Guide](https://python.igraph.org/en/stable/install.html).

### Starting `igraph`

The most common way to use `igraph` is as a named import within an interactive Python environment that you use for exploratory data analysis. This environment can be a bare Python shell, an [IPython](https://ipython.readthedocs.io/) shell, a [Jupyter](https://jupyter.org/) notebook or JupyterLab instance, [Google Colab](https://colab.research.google.com), or an [IDE](https://www.spyder-ide.org/>). In this case, the easiest is to import `igraph` itself under the commonly used `ig` alias:

In [ ]:
import igraph as ig

You can then access all `igraph` functions and variables in the `ig` namespace:

In [ ]:
print(ig.__version__)

If you are using igraph in your own code, an alternative way is simply to import what you need from the igraph package. For instance, if you need the `Graph` class and the `plot` function (more about these later), you can simply import these in your own code and then you do not need the `ig` prefix:

```python
from igraph import Graph, plot

From now on we will assume that you have imported `igraph` into the `ig` namespace; feel free to adjust the code below if you are using a different alias.

### Creating a graph

`igraph` graphs are represented by instances of the `Graph` class. The simplest way to create a graph is by invoking the `Graph` constructor. To make an empty graph:

In [ ]:
g = ig.Graph()

To make a graph with 10 nodes (numbered `0` to `9`) and two edges connecting nodes `0-1` and `0-5`:

In [ ]:
g = ig.Graph(n=10, edges=[[0, 1], [0, 5]])

We can print the graph to get a summary of its nodes and edges:

In [ ]:
print(g)

This means: **U**ndirected graph with **10** vertices and **2** edges, with the exact edges listed out. `igraph` also has a `summary()` function, which is similar to `print()` but it does not list the edges. This is convenient for large graphs with millions of edges:

In [ ]:
ig.summary(g)

### Adding/deleting vertices and edges

Let us start from the empty graph again. To add vertices to an existing graph, we can use the `add_vertices()` method of the graph:

In [ ]:
g = ig.Graph()
g.add_vertices(3)

Due to the C heritage of `igraph`, vertices in a graph are always identified by consecutive integers starting from zero. The number of a vertex is called the *vertex ID*. There are ways to identify vertices by names instead of IDs, but this will be discussed later when we introduce vertex attributes. Right now the three vertices of the graph above are identified with the integers 0, 1 and 2.

THe `add_edges()` method can be used to add edges between vertices that are already added to the graph. For instance, we can add an edge between `0` and `1` and another edge between `1` and `2` as follows:

In [ ]:
g.add_edges([(0, 1), (1, 2)])

Edges are added by specifying the source and target vertex for each edge. Here we refer to the vertices by their numeric IDs, but later on we will learn that we can also use vertex names once we start using vertex attributes.

The graph data structure in `igraph` is optimized for static graphs, i.e. graphs that do not change freqently. If you need to add new vertices or edges to a graph, consider adding them in batches (i.e. by passing multiple edges to a single `add_edges()` call) instead of adding them one by one. `igraph` will do some internal bookkeeping on the data structure after every modification. If you add 1000 edges one by one, these extra operations will be performed 1000 times; on the other hand, if you add 1000 edges in a single batch, this extra cost will have to be paid only once.

Let us now add some more vertices and edges to our graph:

In [ ]:
g.add_edges([(2, 0)])
g.add_vertices(3)
g.add_edges([(2, 3), (3, 4), (4, 5), (5, 3)])
print(g)

We now have an undirected graph with 6 vertices and 7 edges. Vertex and edge IDs are always _continuous_, so if you delete a vertex all subsequent vertices will be renumbered. Deletion of a vertex results in the deletion of its incident edges as well, and since edge IDs also have to be continuous, this means that edge IDs will also change after a vertex deletion (unless you delete an isolated vertex).

You can use the `delete_vertices()` and `delete_edges()` methods of the `Graph` instance to perform these operations. For instance, to delete the edge connecting vertices `2` and `3`, you need to get its ID first, and then you can delete it:

In [ ]:
edge_id = g.get_eid(2, 3)
g.delete_edges(edge_id)

### Graph, vertex and edge attributes

In addition to IDs, vertices and edges can have _attributes_. An attribute can be a name, coordinates for plotting, weights, or any additional metadata that you wish to attach to a vertex or an edge. The graph itself can also have attributes; for instance, a graph may have a name that is then shown when the graph is printed with `print()` or `summary()`. You can treat the entire graph as a mutable mapping that associates attribute names to values, and similarly, the vertex sequence and the edge sequence of the graph is also a mutable mapping that associates attribute names to a list of attribute values, one for each vertex or edge.

To demonstrate the use of attributes, let us create a simple social network:

In [ ]:
names = ["Alice", "Bob", "Claire", "Dennis", "Esther", "Frank", "George"]
edges = [
  ('Alice', 'Bob'),
  ('Alice', 'Claire'),
  ('Claire', 'Dennis'),
  ('Dennis', 'Esther'),
  ('Esther', 'Claire'),
  ('Claire', 'Frank'),
  ('Frank', 'Alice'),
  ('George', 'Dennis'),
  ('Frank', 'George')
]

g = ig.Graph()
g.add_vertices(names)
g.add_edges(edges)

print(g)

What happened here? Earlier, we mentioned that igraph uses nmeric identifiers to refer to vertices and edges, and here we were using _names_ instead. This is possible because all igraph functions that accept numeric vertex identifiers also accept _strings_ instead, and whenever a string is encountered, it is treated as a (most likely unique) name of the vertex and the corresponding vertex ID is looked up automatically behind the scenes. The mapping between vertex names and IDs is stored in a vertex attribute named `"name"`, and this attribute is special in the sense that `igraph` maintains a reverse mapping behind the scenes that allows us to look up the index of a vertex given its name in constant time, as if we were using a Python dictionary.

Therefore, when we called `g.add_vertices()` with a list of strings instead of a single number, igraph automatically created that many new vertices and associated the strings as names to the individual vertices. This also made it possible to call `g.add_edges()` with a list consisting of pairs of vertex _names_, and each name was automatically mapped back to the corresponding vertex ID. Deleting or adding vertices will automatically update the mapping so we do not need to be concerned with the numeric IDs any more if we do not want to -- vertex identities are preserved via the names given to them, and using the same name will always refer to the same vertex even if we delete vertices or add new ones.

Note that `igraph` is pretty strict in the sense that numbers are always treated as vertex IDs and strings are always treated as vertex names. If your graph is built from a dataset where vertices have natural, numeric identifiers, these _cannot_ be used as-is -- you need to convert the numeric identifiers to _strings_ instead.

#### Manipulating attributes with vertex and edge sequences

Vertex attributes can be accessed via the vertex sequence of the graph. Given a graph `g`, the `vs` property stores a reference to the vertex sequence of the entire graph. This object is an instance of a type called `VertexSeq`. Similarly the `es` property stores a reference to the edge sequence, which is an object of type `EdgeSeq`. Both can be used like Python mappings, so you can query the list of vertex names simply by typing:

In [ ]:
g.vs["name"]

Adding new vertex or edge attributes can be done simply by assigning to the keys of the mapping. The assigned value must be a sequence whose length is equal to the number of vertices or edges:

In [ ]:
g.vs["age"] = [25, 31, 18, 47, 22, 23, 50]
g.vs["gender"] = ["f", "m", "f", "m", "f", "m", "m"]
g.es["is_formal"] = [False, False, True, True, True, False, True, False, False]

In fact, the vertex and edge sequences of a graph (i.e. `g.vs` and `g.es`) are both _mappings_ and _sequences_ at the same time. Indexing the vertex sequence with an integer _i_ returns a reference to the vertex with ID equal to _i_, and similarly, indexing the edge sequence with _i_ returns a reference to the edge with ID equal to _i_. Vertices are represented with instances of the `Vertex` class, while edges are represented with instances of `Edge`. Both of these can also be treated as mappings, so you can modify an attribute of an individual vertex or edge by indexing into the vertex or edge sequence first to obtain a `Vertex` or `Edge` object, and then treating the returned object as a mapping to update the attribute value:

In [ ]:
g.es[0]["is_formal"] = True

The entire `Graph` object is also a mutable mapping, providing access to the graph attributes. For instance, to add a date to the graph:

In [ ]:
g["date"] = "2009-01-10"
print(g["date"])

To recap:

* Instances of the `Graph` class can be treated as a mapping that provides access to the graph attributes.
* Each `Graph` instance has a vertex sequence (accessible via the `vs` property) of type `VertexSeq`.
* Instances of `VertexSeq` can be treated as a mapping to gain access to the attributes of all the vertices represented by the sequence.
* Instances of `VertexSeq` can be indexed by integers to gain access to individual vertices of type `Vertex`.
* Similarly each `Graph` instance has an edge sequence (accessible via the `es` property) of type `EdgeSeq`.
* Instances of `EdgeSeq` can be treated as a mapping to gain access to the attributes of all the edges represented by the sequence.
* Instances of `EdgeSeq` can be indexed by integers to gain access to individual edges of type `Edge`.

#### Subsetting vertex and edge sequences

Slicing a `VertexSeq` or an `EdgeSeq` object returns _another_ `VertexSeq` or `EdgeSeq` that represents the vertices and edges of the slice only. For instance, to return the names of every second vertex:

In [ ]:
g.vs[::2]["name"]

Slices can also be used to manipulate attributes only on a subset of vertices or edges. When assigning a new value, you can use a single Boolean, number or string to assign the same value to all matched vertices or edges, or you can use a list to specify an individual value for each matched vertex or edge:

In [ ]:
g.vs[3:5]["foo"] = "eggs"
g.vs[0:3]["foo"] = ("spam", "ham", "bacon")
g.vs["foo"]

Here you can also see that `igraph` automatically adds `None` for unmatched vertices or edges when a new attribute is created via a sub-sequence.

#### Deleting attributes

Attributes can be deleted with the `del` operator, similarly to how you would remove a key from a Python mapping:

In [ ]:
del g.vs["foo"]
assert "foo" not in g.vs


#### Accessing attributes as dictionaries

To retrieve a dictionary of attributes from a `Graph`, `Vertex` or `Edge` object, you can use their `attributes()` method:

In [ ]:
g.vs[0].attributes()

These dictionaries are constructed on-the-fly, therefore modifying them will not affect the attributes of the original object they were retrieved from.

### Layouts and plotting

A graph is an abstract mathematical object without a specific representation in 2D or 3D space. This means that whenever we want to visualise a graph, we have to find a mapping from vertices to coordinates in two- or three-dimensional space first, preferably in a way that is pleasing for the eye. A separate branch of graph theory, namely graph drawing, tries to solve this problem via several graph layout algorithms. `igraph` implements quite a few layout algorithms and is also able to draw them onto the screen or to a PDF, PNG or SVG file. Graph drawing is implemented by separate plotting backends; currently `igraph` supports [Cairo](https://www.cairographics.org/), [Matplotlib](https://matplotlib.org/) and [Plotly](https://plotly.com/). If you want to use Cairo, you need to install `pycairo`, while the Matplotlib and Plotly backends require `matplotlib` and `plotly` from PyPI, respectively. `igraph` tries to determine the appropriate plotting backend automatically based on the set of packages installed in your environment, but for the sake of this tutorial we should be explicit and select `matplotlib` by updating igraph's configuration object as follows:

In [ ]:
ig.config["plotting.backend"] = "matplotlib"

At this point you should also install Matplotlib into your Python environment if you have not done so yet. In a standard Python environment, you can simply execute:

```bash
$ pip install matplotlib
```

In `conda`, you should run this instead:

```bash
$ conda install conda-forge::matplotlib
```

On Google Colab, Matplotlib is part of the default environment so you do not need to do anything extra.

If you are following this tutorial in JupyterLab or VS Code, you might have to restart the kernel and execute all cells again when you reach this point to make sure that `igraph` recognizes the newly added `matplotlib` library in your environment.

We can now try plotting our graph using the `plot()` function from the `ig` namespace:

In [ ]:
ig.plot(g)

With the Matplotlib backend, the `plot()` function returns a Matplotlib artist. If you are running this function in a notebook environment like Jupyter, VS Code or Google Colab, you will get the visualization of the graph directly within the notebook. To plot the graph onto an existing Matplotlib figure, you need to use the `target=...` keyword argument of the `plot()` function like this:

```python
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ig.plot(g, target=ax)
```

#### Calculating the layout of a plot

igraph contains several built-in layout algorithms, accessible as methods of the `Graph` object. The names of all layout methods start with `layout_...`. The following table summarizes the available layouts:

| Method name | Short name | Description |
| :---------- | :--------- | :---------- |
| `layout_circle` | `circle`, `circular` | Deterministic layout that places the vertices on a circle |
| `layout_davidson_harel` | `dh` | Davidson-Harel simulated annealing algorithm |
| `layout_drl` | `drl` | The [Distributed Recursive Layout](https://www.osti.gov/doecode/biblio/54626) algorithm for large graphs |
| `layout_fruchterman_reingold` | `fr` | Fruchterman-Reingold force-directed algorithm |
| `layout_fruchterman_reingold_3d` | `fr3d`, `fr_3d` | Fruchterman-Reingold force-directed algorithm in three dimensions |
| `layout_graphopt` | `graphopt` | The GraphOpt algorithm for large graphs |
| `layout_grid` | `grid` | Regular grid layout |
| `layout_kamada_kawai` | `kk` | Kamada-Kawai force-directed algorithm |
| `layout_kamada_kawai_3d` | `kk3d`, `kk_3d` | Kamada-Kawai force-directed algorithm in three dimensions |
| `layout_lgl` | `large`, `lgl`, `large_graph` | The [Large Graph Layout](https://sourceforge.net/projects/lgl/) algorithm for large graphs |
| `layout_mds` | `mds` | Multidimensional scaling layout |
| `layout_random` | `random` | Places the vertices completely randomly |
| `layout_random_3d` | `random_3d` | Places the vertices completely randomly in 3D |
| `layout_reingold_tilford` | `rt`, `tree` | Reingold-Tilford tree layout, useful for (almost) tree-like graphs |
| `layout_reingold_tilford_circular` | `rt_circular` | Reingold-Tilford tree layout with a polar coordinate post-transformation, useful for (almost) tree-like graphs |
| `layout_sphere` | `sphere`, `spherical`, `circular_3d` | Deterministic layout that places the vertices evenly on the surface of a sphere |

Layout algorithms can either be called directly or using the common layout method named `layout()`:

In [ ]:
layout = g.layout_kamada_kawai()
layout = g.layout("kamada_kawai")
print(layout)


The first argument of the `layout()` method must be the short name of the layout algorithm (see the table above). All the remaining positional and keyword arguments are passed intact to the chosen layout method. For instance, the following two calls are completely equivalent:

In [ ]:
layout = g.layout_reingold_tilford(root=[2])
layout = g.layout("rt", root=[2])

#### Drawing a graph with a given layout

Layout methods return a `Layout` object, which behaves mostly like a list of lists. Each list entry in a `Layout` object corresponds to a vertex in the original graph and contains the vertex coordinates in the 2D or 3D space. `Layout` objects also contain some useful methods to translate, scale or rotate the coordinates in a batch. However, the primary utility of `Layout` objects is that you can pass them to the `plot()` function along with the graph to obtain a 2D drawing. For instance, we can plot our imaginary social network with the Kamada-Kawai layout algorithm as follows:

In [ ]:
layout = g.layout("kk")
ig.plot(g, layout=layout)

When no layout is specified explicitly, `igraph` falls back to a special layout algorithm implemented in the `layout_auto()` method, which selects one of the other layout algorithms based on the size of the graph. This layout can also be invoked explicitly if needed.

#### Decorating a layout with labels and colors

The layouts we have seen so far are not too pretty. A trivial addition would be to use the names as the vertex labels and to color the vertices according to the gender. Vertex labels are taken from the `label` attribute by default and vertex colors are determined by the `color` attribute, so we can simply create these attributes and re-plot the graph. To preserve the layout from the previous figure, we re-use it from the `layout` variable (otherwise we would get a new layout every time the plot is created):

In [ ]:
g.vs["label"] = g.vs["name"]
g.vs["color"] = ["skyblue" if gender == "m" else "pink" for gender in g.vs["gender"]]
ig.plot(g, layout=layout)

We can also move the labels outside the vertices using the `label_dist` vertex attribute. This attribute takes a number and moves the label outwards from the center of the node by the size of the vertex multiplied by this value. We can also change the colors of the edges to a lighter gray to improve the readability of the labels:

In [ ]:
g.vs["label_dist"] = 1.25
g.es["color"] = "gray"
ig.plot(g, layout=layout)

In practice, it is often a good idea to keep the visual representation of a graph separate from its layout. This can be done by storing the visualization properties in a Python dict. In this dict, the keys of vertex-related visualization attributes must be prefixed with `vertex_` and edge-related attributes must be prefixed with `edge_`, like this:

In [ ]:
visual_style = {
    "vertex_size": 20,
    "vertex_color": ["skyblue" if gender == "m" else "pink" for gender in g.vs["gender"]],
    "vertex_label": g.vs["name"],
    "edge_width": [1 + 2 * int(is_formal) for is_formal in g.es["is_formal"]],
    "layout": layout
}

The dictionary can then be spread into keyword arguments of the `plot()` function, and the values prescribed here will take precedence over visualization-related attributes of vertices and edges. Note that in the above configuration, we also assigned thicker edges to formal connections between people in the network. Hoever, since we have not specified edge colors explicitly, they are still taken from the `width` edge attribute:

In [ ]:
ig.plot(g, **visual_style)

With the Matplotlib backend, you can also use color maps straight from Matplotlib to set the colors of vertices and edges based on the value of some continuous property. In the followig example, we are going to color the vertices based on their betweenness centrality by normalizing the centralities to the [0; 1] range first and then feeding them through a Matplotlib linear color map:

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

cmap = LinearSegmentedColormap.from_list("vertex_cmap", ["white", "red"])
visual_style["vertex_color"] = [cmap(x) for x in ig.rescale(g.betweenness())]
print(visual_style["vertex_color"])

ig.plot(g, **visual_style)

The following tables summarize all the graph, vertex and edge related attributes that can be used to change the visual appearance of the graph:

##### Vertex attributes controlling graph plots

| Attribute name | Keyword argument | Purpose |
| :------------- | :--------------- | :------ |
| `color`        | `vertex_color`   | Color of the vertex |
| `frame_color`  | `vertex_frame_color` | Stroke color of the vertex shape |
| `frame_width`  | `vertex_frame_width` | Stroke width of the vertex shape |
| `font`         | `vertex_font`    | Font family of the vertex |
| `label`        | `vertex_label`   | Label of the vertex |
| `label_angle`  | `vertex_label_angle` |  The placement of the vertex label on the circle around the vertex. This is an angle in radians, with zero belonging to the right side of the vertex. |
| `label_color`  | `vertex_label_color` | Color of the vertex label |
| `label_dist`   | `vertex_label_dist` | Distance of the vertex label from the vertex itself, relative to the vertex size |
| `label_size`   | `vertex_label_size` | Font size of the vertex label |
| `order`        | `vertex_order` | Drawing order of the vertices. Vertices with a smaller order parameter will be drawn first |
| `shape`        | `vertex_shape` | Shape of the vertex. Known shapes are: `rectangle`, `circle`, `diamond`, `hidden`, `triangle-up`, `triangle-down` |
| `size`         | `vertex_size` | Size of the vertex in pixels |

##### Edge attributes controlling graph plots

| Attribute name | Keyword argument | Purpose |
| :------------- | :--------------- | :------ |
| `color`        | `edge_color`     | Color of the edge |
| `curved`       | `edge_curved`    | The curvature of the edge. Positive values correspond to edges curved in CCW direction, negative numbers correspond to edges curved in clockwise (CW) direction. Zero represents straight edges. `True` is interpreted as 0.5, `False` is interpreted as zero. This is useful to make multiple edges visible. See also the `autocurve` keyword argument to `plot()` |
| `font`         | `edge_font`      | Font family of the edge |
| `arrow_size`   | `edge_arrow_size` | Size (length) of the arrowhead on the edge if the graph is directed, relative to 15 pixels. |
| `arrow_width`  | `edge_arrow_width` | Width of the arrowhead on the edge if the graph is directed, relative to 10 pixels. |
| `loop_size`    | `edge_loop_size` | Size of self-loops. It can be set as a negative number, in which case it scales with the size of the corresponding vertex (e.g. -1.0 means the loop has the same size as the vertex). This attribute is ignored by edges that are not loops. |
| `width`        | `edge_width` | Width of the edge in pixels. |
| `label`        | `edge_label` | If specified, it adds a label to the edge. |
| `background`   | `edge_background` | If specified, it adds a rectangular box around the edge label, of the specified color. |
| `align_label`  | `edge_align_label` | If `True`, rotate the edge label such that it aligns with the edge direction. Labels that would be upside-down are flipped. |

#### Saving plots

`igraph` plots using the Matplotlib backend can be saved just like any other Matplotlib figure (and in fact you can even combine Matplotlib plots with graphs drawn by `igraph`). For instance, you can generate a geometric random graph and plot it along with its degree distribution side-by-side such that the graph plot is provided by igraph and the degree distribution plot is provided by Matplotlib:

In [ ]:
from matplotlib import pyplot as plt

g = ig.Graph.GRG(1000, 0.05)
layout = ig.Layout(zip(g.vs["x"], g.vs["y"]))

fig, axs = plt.subplots(1, 2)
ig.plot(g, layout=layout, target=axs[0], vertex_size=5, edge_width=0.5)
plt.hist(g.degree())

Once the plot is ready, you can simply call `plt.savefig()` to save the current figure to a file:

```python
plt.savefig("figure.pdf")
```

### Exporting, importing and converting graphs

No graph module would be complete without some kind of import/export functionality that enables the package to communicate with external programs and toolkits. igraph is no exception: it provides functions to read the most common graph formats and to save Graph objects into files obeying these format specifications. The following table summarizes the formats igraph can read or write:

| Format | Short name | Reader method | Writer method |
| :----- | :--------- | :------------ | :------------ |
| Adjacency list (a.k.a. [LGL](https://lgl.sourceforge.net/#FileFormat)) | `lgl` | `Graph.Read_Lgl()` | `Graph.write_lgl()` |
| Adjacency matrix | `adjacency` | `Graph.Read_Adjacency()` | `Graph.write_adjacency()` |
| DIMACS | `dimacs` | `Graph.Read_DIMACS()` | `Graph.write_dimacs()` |
| DL | `dl` | `Graph.Read_DL()` | _not supported yet_ |
| Edge list | `edgelist`, `edges`, `edge` | `Graph.Read_Edgelist()` | `Graph.write_edgelist()` |
| GraphViz | `graphviz`, `dot` | _not supported yet_ | `Graph.write_dot()` |
| GML | `gml` | `Graph.Read_GML()` | `Graph.write_gml()` |
| GraphML | `graphml` | `Graph.Read_GraphML()` | `Graph.write_graphml()` |
| Gzipped GraphML | `graphmlz` | `Graph.Read_GraphMLz)` | `Graph.write_graphmlz()` |
| LEDA | `leda` | _not supported yet_ | `Graph.write_leda()` |
| Labeled edgelist (a.k.a. [NCOL](https://lgl.sourceforge.net/#FileFormat)) | `ncol` | `Graph.Read_Ncol()` | `Graph.write_ncol()` |
| Pajek | `pajek`, `net` | `Graph.Read_Pajek()` | `Graph.write_pajek()` |
| Pickled | `pickle` | `Graph.Read_Pickle()` | `Graph.write_pickle()` |

As an exercise, download the graph representation of the well-known [Zachary karate club study](https://en.wikipedia.org/wiki/Zachary%27s_karate_club) from [this file](zachary.graphml), and try to load it into `igraph`. Since it is a GraphML file, you must use the GraphML reader method from the table above (make sure you use the appropriate path to the downloaded file):

In [ ]:
karate = ig.Graph.Read_GraphML("zachary.graphml")
print(karate)

If you want to convert this graph into, say, Pajek's format, you can do it
with the Pajek writer method from the table above:

```python
karate.write_pajek("zachary.net")
```


Most of the formats have their own limitations; for instance, not all of them can store attributes. Your best bet is probably GraphML or GML if you want to save `igraph` graphs in a format that can be read from an external package and you want to preserve numeric and string attributes. Edge list and NCOL is also fine if you don't have attributes (NCOL supports vertex names and edge weights, though). If you don't want to use your graphs outside `igraph` but you want to store them for a later session, the pickled graph format ensures that you get exactly the same graph back. The pickled graph format uses Python's `pickle` module to store and read graphs.

There are two helper methods as well: `read()` is a generic entry point for reader methods which tries to infer the appropriate format from the file extension. `write()` is the opposite of `read()`: it lets you save a graph where the preferred format is again inferred from the extension. The format detection logic of `read()` and `write()` can be overridden by the ``format`` keyword argument, which accepts the short names of the formats from the above table:

```python
karate = ig.load("zachary.graphml")
karate.write("zachary.net")
karate.write("zachary.my_extension", format="gml")
```

### Interfacing with other network science packages

Exporting an `igraph` graph into a file and then importing it into your favourite network science package is the most generic way to get `igraph` work together with some other tool that you use in your research. However, if you are a lucky user of NetworkX or `graph-tool`, you can convert igraph graphs to/from NetworkX and `graph-tool` without having to use an on-disk representation.

Conversion to NetworkX is achievable with the `from_networkx()` and `to_networkx()` functions:

In [ ]:
from networkx import Graph

karate_nx = karate.to_networkx()
print(karate_nx)
if isinstance(karate_nx, Graph):
    print("Yay, this is indeed a NetworkX graph!")

Conversion preserves vertex and edge attributes as well, and of course you can also do the reverse conversion:

In [ ]:
karate_ig = ig.Graph.from_networkx(karate_nx)
if karate_ig.get_edgelist() == karate.get_edgelist():
    print("Conversion roundtrip to NetworkX succeeded")

### Example: Significance of community structure in networks

In [ ]:
def rewired(g):
    """Given an undirected graph g, returns a random graph that has exactly the
    same degree sequence as the original graph.
    """
    return ig.Graph.Degree_Sequence(g.degree(), method="edge_switching_simple")


def max_modularity(g):
    """Given a graph g, returns the modularity of its best partition using the
    fast greedy modularity optimization algorithm.
    """
    return g.community_fastgreedy().as_clustering().modularity


def significance(g, measure, *, n=500):
    """Given an undirected graph g and a graph measure (a function that can be
    called with a graph and returns a numeric value), returns the probability
    of observing a value that is larger than or equal to the measure calculated
    on the graph itself among n realizations of random graphs with the same
    degree sequence.
    """
    reference = measure(g)
    randomized = [measure(rewired(g)) for _ in range(n)]
    return sum(x > reference for x in randomized) / n


print("BA graph:", significance(ig.Graph.Barabasi(1000, m=2), max_modularity))
print("GR graph:", significance(ig.Graph.GRG(1000, 0.05), max_modularity))


### Example: Fast algorithm prototyping

[Clique percolation method](https://en.wikipedia.org/wiki/Clique_percolation_method): Palla et al, Uncovering the overlapping community structure of complex networks in nature and society. Nature **435**:814–818, 2005.
 
![Illustration of the clique percolation method](Illustration_of_overlapping_communities.svg)

In [ ]:
from itertools import combinations


def cpm(g, k=4):
    cliques = [set(clique) for clique in g.maximal_cliques(min=k)]
    edges = [
        (i, j)
        for i, j in combinations(range(len(cliques)), 2)
        if len(cliques[i] & cliques[j]) >= k-1
    ]
    components_in_clique_graph = ig.Graph(edges).connected_components()
    communities = [
        sorted(set().union(*[cliques[i] for i in component]))
        for component in components_in_clique_graph
    ]
    return ig.VertexCover(g, communities)


g = ig.Graph.GRG(100, 0.15)
print(cpm(g))